# Análise das Stories

Notebook para o auxílio da análise das stories do chatbot.

### Configurando e Imports

In [1]:
import rasa_nlu
import rasa_core
from IPython.display import IFrame

print("Versões utilizadas neste jupyter notebook:")
print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))


Versões utilizadas neste jupyter notebook:
rasa_nlu: 0.14.1 rasa_core: 0.13.1


## Análise e Avaliação das Stories

### Visualizaçõa do Fluxo de Conversa

O comando abaixo monta um grafo com a relação das `intents` e `utters` do chatbot, ou seja, você vai conseguir visualizar o fluxo de conversa do seu chatbot.

Esta visualização é importante para verificar possíveis problemas na estrutura do seu chatbot e se ele realmente chega em determinadas "**falas**" da conversa.

In [2]:
!python -m rasa_core.visualize -d $BOT_DOMAIN_PATH -s $BOT_STORIES_PATH -o chat_graph.html

/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2019-05-03 16:56:08 INFO     __main__  - Starting to visualize stories...
Processed Story Blocks: 100%|███| 34/34 [00:00<00:00, 7531.76it/s, # trackers=1]
2019-05-03 16:56:08 INFO     __main__  - Finished graph creation. Saved into file:///work/notebooks/stories/chat_graph.html


O Rasa gerou uma página `html` com o grafo de conversa, para facilitar, você pode visualizar o grafo no arquivo `chat_graph.html` aqui na próxima celula.

Use o mouse para dar zoom e arrastar sobre o conteúdo do grafo. Altere `width` e `height` se desejar.

* Dica: caso você se perca com **zoom in** ou **zoom out** no grafo, apeas re-execute a célula abaixo e ele irá reaparecer na célula.

In [3]:
IFrame(src='./chat_graph.html', width=900, height=700)

### Avaliação das Stories

Outra forma de analisar seu chatbot é por meio da própria avaliação do Rasa, ele gera uma matriz de confuzão com os dados fornecidos nas `stories` e do resultado do treinamento armazenado na pasta `models`.

* Caso você ainda não tenha treinado seu chatbot execute a céclula abaixo para treina-lo.

In [4]:
!make train -C $BOT_DIR_PATH

make: Entering directory '/work/bot'
python3 -m rasa_nlu.train -c nlu_config.yml --fixed_model_name current \
  --data data/intents/ -o models --project nlu --verbose
/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2019-05-03 16:56:11 INFO     rasa_nlu.training_data.loading  - Training data format of data/intents/aleatorio.md is md
2019-05-03 16:56:11 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 197 (24 distinct intents)
	- Found intents: 'como_estou', 'piada', 'license', 'cor', 'me', 'bff', 'filhos', 'esporte', 'risada', 'hobby', 'relationship', 'linguagens', 'onde_voce_mora', 'filme', 'time', 'historia', 'signo', 'playlist', 'de_onde_voce_eh', 'triste', 'star_wars', 'genero', 'religiao', 'comida'
	- entity examples: 172 (22 distinct entities)
	- found entities: 'license', 'piada', 'live', 'relation

Epoch 1/8
2034/2034 [==============================] - 2s 1ms/step - loss: 2.5072 - acc: 0.5529
Epoch 2/8
2034/2034 [==============================] - 2s 940us/step - loss: 1.5712 - acc: 0.6543
Epoch 3/8
2034/2034 [==============================] - 2s 931us/step - loss: 1.0025 - acc: 0.7822
Epoch 4/8
2034/2034 [==============================] - 2s 746us/step - loss: 0.5209 - acc: 0.9162
Epoch 5/8
2034/2034 [==============================] - 2s 976us/step - loss: 0.2789 - acc: 0.9562
Epoch 6/8
2034/2034 [==============================] - 2s 883us/step - loss: 0.1835 - acc: 0.9665
Epoch 7/8
2034/2034 [==============================] - 1s 723us/step - loss: 0.1304 - acc: 0.9756
Epoch 8/8
2034/2034 [==============================] - 2s 894us/step - loss: 0.1050 - acc: 0.9809
2019-05-03 16:56:49 INFO     rasa_core.policies.keras_policy  - Done fitting keras policy model
2019-05-03 16:56:49 DEBUG    rasa_core.featurizers  - Creating states and action examples from collected trackers (by MaxH

* `rasa_core.evaluate` é o método utilizado para verificar a qualidade das suas `stories`.

In [ ]:
!python -m rasa_core.evaluate --core $BOT_DIR_PATH'models/dialogue' -s $BOT_STORIES_PATH -o results

/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2019-05-03 16:57:01.773529: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


* Caso algum problme seja encotrado ele será descrito no arquivo `failed_stories.md`

In [ ]:
!cat results/failed_stories.md

* Se tudo estiver correto você deverá ver a mensagem:

`<!-- All stories passed -->`

* Outro arquivo gerado é  o `story_confmat.pdf` uma matriz de confuzão onde é possível visualizar a relação entre as `stories`.

In [ ]:
from IPython.display import IFrame
IFrame("results/story_confmat.pdf", width=900, height=900)

## Referências:

O Rasa está em constante evolução, alguns links úteis para a construção deste jupyter-notebook e para a análise das `stories` são:

* [Evaluation](https://rasa.com/docs/core/evaluation/)
* [Debugging](https://rasa.com/docs/core/debugging/)